In [2]:
"""Getting all relevent imports into the file"""

import pandas as pd #imported pandas to use dataframes
import numpy as np #imported numpy for dynamic arrays
import matplotlib.pyplot as plt # imported pyplot to use with my visuals
import requests #imported to use webscrape
from requests import get #pulled get to use for scraping
from random import randint #pulled randint to use as a buffer for time
from time import sleep #pulled sleep to slow the crawl rate of the scraper
from bs4 import BeautifulSoup #pulled to webscrape


In [51]:
"""Pulling information from the actual IMDb website."""
pages = np.arange(1,1001,250)
title = []
movie_rating = []
year =  []
star_rating = []
gross = []
movie_genre = []
movie_time = []
votes = []
def scraper():
    for page in pages:
        page = requests.get('https://www.imdb.com/search/title/?release_date=1700-01-01,2020-12-31&groups=top_1000&certificates=US%3AG,US%3APG,US%3APG-13,US%3AR,US%3ANC-17&sort=alpha,asc&count=250&start='+str(page)+'&ref_=adv_nxt')
        
        soup = BeautifulSoup(page.text, 'html.parser')
        
#pulling from lister-item mode-advanced pulls all the information for each of the movies on the page to easily pull 
#the Html tags as attributes of an item
        container = soup.findAll('div',class_ = 'lister-item mode-advanced') 
        """ for loop looking for each of the catagories in each of the containers"""
        for name in container:
            """nv is so we can seperate the gross income and seperate it from the votes set into their own """
            nv = name.find_all('span',attrs ={'name':'nv'})
            
            title.append(name.h3.a.text)
            movie_rating.append(name.p.span.text)
            year.append(name.h3.find('span',class_ = 'lister-item-year').text) 
            star_rating.append(float(name.strong.text))
            gross_income = nv[1].text.strip('$\n\tM') if len(nv) > 1 else '0'
            gross.append(gross_income)
            votes_total = nv[0].text if len(nv) >1 else '0'
            votes.append(votes_total)
            movie_genres = name.p.find('span',class_ ='genre').text.strip(',\n\t" "').split(',')
            #movie_genre.append(list(map(lambda ele: ele.split(','),movie_genres)))#movie_genres
            movie_genre.append(movie_genres)
            movie_time.append(name.p.find('span', class_ = 'runtime').text)


In [52]:
"""Cleaning of the information. removing unneeded characters out of the set of information as well as setting up the dataframe"""
def Clean():
    df.loc[:,'Year'] = df['Year'].astype(str).str[-5:-1].astype(int)
    df.loc[:,'Movie Runtime'] = df['Movie Runtime'].astype(str).str[:-3].astype(int)
    df.loc[:,'Votes'] = df['Votes'].str.replace(',','').astype(float)
    df.loc[:,'Gross Income']= df['Gross Income']
    df.loc[:, 'Title'] = df['Title'].astype(str)
    df.loc[:,'Movie Genre'] = df['Movie Genre'].astype(str)
    df.drop_duplicates()
    

In [53]:
"""Pull and clean information to get ready for setting to dataframe"""
scraper()


In [54]:
"""Setting the information from above to a dataframe to manipulate and visualize"""
df = pd.DataFrame({'Title': title,'Movie Rating': movie_rating,'Year': year,
                   'Gross Income': gross,'Movie Genre': movie_genre, 'Movie Runtime': movie_time,
                    'IMDb Rating': star_rating,'Votes': votes})
Clean()
df.head()

,Title,Movie Rating,Year,Gross Income,Movie Genre,Movie Runtime,IMDb Rating,Votes
0,500 Days of Summer,PG-13,2009,32.39,"['Comedy', ' Drama', ' Romance']",95,7.7,461982.0
1,12 Years a Slave,R,2013,56.67,"['Biography', ' Drama', ' History']",134,8.1,620081.0
2,127 Hours,R,2010,18.34,"['Biography', ' Drama']",94,7.6,340547.0
3,1917,R,2019,159.23,"['Drama', ' War']",119,8.3,341756.0
4,2001: A Space Odyssey,G,1968,56.95,"['Adventure', ' Sci-Fi']",149,8.3,583101.0


In [55]:
df.set_index('Title')

,Movie Rating,Year,Gross Income,Movie Genre,Movie Runtime,IMDb Rating,Votes
Title,,,,,,,
500 Days of Summer,PG-13,2009,32.39,"['Comedy', ' Drama', ' Romance']",95,7.7,461982.0
12 Years a Slave,R,2013,56.67,"['Biography', ' Drama', ' History']",134,8.1,620081.0
127 Hours,R,2010,18.34,"['Biography', ' Drama']",94,7.6,340547.0
1917,R,2019,159.23,"['Drama', ' War']",119,8.3,341756.0
2001: A Space Odyssey,G,1968,56.95,"['Adventure', ' Sci-Fi']",149,8.3,583101.0
...,...,...,...,...,...,...,...
Cold War,R,2018,4.57,"['Drama', ' History', ' Music']",89,7.6,44913.0
Zodiac,R,2007,33.08,"['Crime', ' Drama', ' Mystery']",157,7.7,437580.0
Zombieland,R,2009,75.59,"['Adventure', ' Comedy', ' Fantasy']",88,7.6,507644.0


In [ ]:
df.to_csv("df.csv",index=False)

In [64]:
df_budget=pd.read_csv('tn.movie_budgets.csv.gz') # get the data for budget of movies
df_budget.rename(columns={'movie': 'Title'}, inplace=True) # rename the movie column to title so it can be used as an index for joining 
df_budget.set_index('Title',inplace=True) # set the name as index 


In [16]:
df_budget.head()

,id,release_date,production_budget,domestic_gross,worldwide_gross
Title,,,,,
Avatar,1,"Dec 18, 2009","$425,000,000","$760,507,625","$2,776,345,279"
Pirates of the Caribbean: On Stranger Tides,2,"May 20, 2011","$410,600,000","$241,063,875","$1,045,663,875"
Dark Phoenix,3,"Jun 7, 2019","$350,000,000","$42,762,350","$149,762,350"
Avengers: Age of Ultron,4,"May 1, 2015","$330,600,000","$459,005,868","$1,403,013,963"
Star Wars Ep. VIII: The Last Jedi,5,"Dec 15, 2017","$317,000,000","$620,181,382","$1,316,721,747"


In [65]:
df_budget['production_budget'] = df_budget['production_budget'].apply(lambda x: x.strip('$').replace(",","")) #Clean currency string  
df_budget['production_budget'] = df_budget['production_budget'].astype(float)/1000000 # Convert to integer and change the amount into millions

In [66]:
type(df_budget['production_budget'][0])# check to see if amount is numeric

numpy.float64

In [67]:
df=pd.read_csv('df.csv')

In [68]:
fulldf = pd.merge(df,df_budget['production_budget'],on ='Title') # add the budget column and release date to df

In [69]:
fulldf.head()

,Title,Movie Rating,Year,Gross Income,Movie Genre,Movie Runtime,IMDb Rating,Votes,production_budget
0,12 Years a Slave,R,2013,56.67,"['Biography', ' Drama', ' History']",134,8.1,620081.0,20.0
1,127 Hours,R,2010,18.34,"['Biography', ' Drama']",94,7.6,340547.0,18.0
2,2001: A Space Odyssey,G,1968,56.95,"['Adventure', ' Sci-Fi']",149,8.3,583101.0,10.5
3,21 Grams,R,2003,16.29,"['Crime', ' Drama', ' Thriller']",124,7.7,220464.0,20.0
4,25th Hour,R,2002,13.06,['Drama'],135,7.6,167274.0,4.5


In [85]:
fulldf.to_csv("fulldf.csv",index=False)

In [105]:
#dff=df.groupby('Movie Genre').count()
#print(dff[dff.Title == dff.Title.max()]) 

             Title  Movie Rating  Year  Gross Income  Movie Runtime  \
Movie Genre                                                           
['Drama']       62            62    62            62             62   

             IMDb Rating  Votes  
Movie Genre                      
['Drama']             62     62  
